In [41]:
# Import dependencies

import pandas as pd
import requests
import gmaps
from config import g_key



In [42]:
gmaps.configure(api_key=g_key)

In [24]:
weather_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
weather_df.head()

,City ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,288.64,71,56,3.71,broken clouds
1,1,Castro,BR,-24.7911,-50.0119,287.77,95,18,2.68,few clouds
2,2,Busselton,AU,-33.6500,115.3333,286.94,86,0,3.56,clear sky
3,3,Zhanaozen,KZ,43.3000,52.8000,284.01,52,70,0.61,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,286.19,74,5,1.01,clear sky


In [25]:
weather_F_conversion = [(temp-273.15) * (9/5) + 32 for temp in weather_df['Max Temp']]


In [26]:
weather_df.loc[:, 'Max Temp'] = weather_F_conversion

weather_df.head()

,City ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.882,71,56,3.71,broken clouds
1,1,Castro,BR,-24.7911,-50.0119,58.316,95,18,2.68,few clouds
2,2,Busselton,AU,-33.6500,115.3333,56.822,86,0,3.56,clear sky
3,3,Zhanaozen,KZ,43.3000,52.8000,51.548,52,70,0.61,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,55.472,74,5,1.01,clear sky


In [27]:
min_temp = float(input('What is the minimum preferable temperature for your trip?'))
max_temp = float(input('What is the maximum preferable temperature for your trip?'))

In [46]:
weather_pref_df = weather_df.loc[(weather_df['Max Temp'] >= min_temp) & \
                                 (weather_df['Max Temp'] <= max_temp)]



In [47]:
weather_pref_df.count()

City ID                205
City                   205
Country                205
Lat                    205
Long                   205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [48]:
weather_pref_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Long                   0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [52]:
hotel_df = weather_pref_df[['City', 'Country', 'Max Temp', 'Lat', 'Long']].copy()

hotel_df['Hotel Name'] = ''


In [53]:
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [54]:
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lon = row['Long']

    params['location'] = f'{lat},{lon}'

    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    hotels = requests.get(url, params=params).json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']

    except (IndexError):
        print('Hotel not found...skipping.')


Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [55]:
hotel_df.head()

,City,Country,Max Temp,Lat,Long,Hotel Name
6,Cayenne,GF,82.094,4.9333,-52.3333,Hôtel Le Dronmi
13,Vaini,TO,84.362,-21.2000,-175.2000,Keleti Beach Resort
17,Manaure,CO,87.566,11.7751,-72.4445,Hotel Palaaima
21,Ibra,OM,77.558,22.6906,58.5334,مزرعة راحة النفوس المنزفة
22,Beira,MZ,78.890,-19.8436,34.8389,Beira Terrace Hotel


In [63]:
hotel_df.isnull().sum()

City          0
Country       0
Max Temp      0
Lat           0
Long          0
Hotel Name    0
dtype: int64